Both prophet and plotly need to be installed on machine.

In [1]:
import pandas as pd
from prophet import Prophet

In [3]:
df = pd.read_csv("Twitter_May_Aug_2014_TerrorSecurity_resolved.txt", sep=" ", header=None)
df.columns = ['dateTime', 'mentionedEntity1', 'mentionedEntity2']
df

,dateTime,mentionedEntity1,mentionedEntity2
0,05:02:2014:20:15:22,statusid_462321880817745921,userid_831528001
1,05:02:2014:20:15:22,statusid_462321880817745921,userid_11134252
2,05:02:2014:20:15:22,statusid_462321880817745921,userid_20733972
3,05:02:2014:20:15:22,statusid_462321880817745921,benghazi
4,05:02:2014:20:15:22,statusid_462321880817745921,gop_benghazi
...,...,...,...
2601829,08:01:2014:04:19:58,userid_1271158584,chiltern_firehouse
2601830,08:01:2014:04:19:58,userid_1271158584,london
2601831,08:01:2014:04:19:58,userid_1271158584,july
2601832,08:01:2014:04:19:58,userid_1537804958,http://pbs.twimg.com/media/bt7phzcceaa7bm5.jpg
